In [1]:
from tictactoe import TicTacToeEnv
import random
import gym

from keras.models import Sequential
from keras import layers
from keras import losses
from keras import optimizers
import numpy as np

In [2]:
#from keras.model import Sequential

env = gym.make('tictactoe-v0')

def create_agent_model():
    model = Sequential()

    # each position can be empty, 0 or 1 and there are 9 positions
    # we encode each position with two 0/1 values.  empty = 0/0, player1 = 1/0, player2 = 0/1
    model.add(layers.Dense(18, activation="relu"))
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(9))
    
    optimizer =  optimizers.Adam(learning_rate=0.01)
    loss = losses.MeanSquaredError()
    model.compile(optimizer, loss)
    
    return model


class Agent:
    def __init__(self, model):
        self.model = model
        self.env = env
        self.use_randomness = False
        self.random_rate = 0.2
        
    def set_random_rate(self, rate):
        self.random_rate = rate
        
    def get_action(self, env):
        if self.use_randomness:
            return random.randint(0,8)
        else:
            best_action = self._get_best_action()
            return best_action
    
    def _get_best_action(self):
        best_action = np.argmax(self.model.predict(np.array([env._one_hot_board()])))
        return best_action
        
    def _get_random_action(self):
        # truly random, so could yield invalid moves
        return random.randint(0, 8)
    
    def set_use_randomness(self, value):
        self.use_randomness = value
    
    #TODO: get random *valid* action and get best *valid* action (to avoid invalid moves when not training)

In [3]:
def record_experience(env, agent):
    # plays a game until it's done, recording all steps into tuples (state, action, reward, next_state, done)
    env.reset()
    done = False
    experience = []
    while not done:
        action = agent.get_action(env)
        state = env._one_hot_board()
        step_result = env.step(action)
        (next_state, reward, done, info) = step_result
        
        experience.append((state, action, reward, next_state, done))
    return experience

In [4]:
model = create_agent_model()
agent = Agent(model)

In [5]:

#  work in progress

def train_model(experiences, model, verbose=False):
    start_states = np.array([e[0] for e in experiences])
    actions = np.array([e[1] for e in experiences])
    rewards = np.array([e[2] for e in experiences])
    next_states = np.array([e[3] for e in experiences])
    predictions  = model.predict(start_states)
    next_predictions = model.predict(next_states)
    
    gamma = 0.95
    for prediction,action,reward,next_prediction in zip(predictions, actions, rewards, next_predictions):
        prediction[action] = reward + gamma * np.max(next_prediction)
    
    model.fit(x=start_states, y=predictions, epochs=1, verbose=verbose)
    
def train_game_per_game(num_games, env, agent):
    for i in range(num_games):
        if i%100 == 0:
            experience = record_experience(env, agent)
            train_model(experience, model, verbose=True)
            print("round %d, game duration %d"%(i, len(experience)))
        else:
            experience = record_experience(env, agent)
            train_model(experience, model)
            
def train_batch_experiences(num_actions, env, agent, verbose=False):
    env.reset()
    done = False
    experiences = []
    for i in range(num_actions):
        action = agent.get_action(env)
        state = env._one_hot_board()
        
        step_result = env.step(action)
        (next_state, reward, done, info) = step_result
        
        experiences.append((state, action, reward, next_state, done))
        
        if done:
            env.reset()
      
    print("training with %d experiences"%(len(experiences)))
    train_model(experiences, model, verbose)

agent.set_use_randomness(True)
train_game_per_game(10000, env, agent)
#for epoch in range(5000):
#    print("epoch", epoch)
#    train_batch_experiences(100, env, agent, True)

1/1 [==============================] - 0s 1ms/step - loss: 0.0139
round 0, game duration 5
1/1 [==============================] - 0s 745us/step - loss: 2.9503
round 100, game duration 5
1/1 [==============================] - 0s 739us/step - loss: 2.3304
round 200, game duration 3
1/1 [==============================] - 0s 728us/step - loss: 2.1806
round 300, game duration 3
1/1 [==============================] - 0s 683us/step - loss: 1.5559
round 400, game duration 5
1/1 [==============================] - 0s 678us/step - loss: 2.0525
round 500, game duration 4
1/1 [==============================] - 0s 755us/step - loss: 0.5861
round 600, game duration 5
1/1 [==============================] - 0s 754us/step - loss: 0.1100
round 700, game duration 2
1/1 [==============================] - 0s 666us/step - loss: 0.5485
round 800, game duration 4
1/1 [==============================] - 0s 668us/step - loss: 1.4242
round 900, game duration 3
1/1 [==============================] - 0s 703us/step -

1/1 [==============================] - 0s 870us/step - loss: 344.4851
round 8500, game duration 5
1/1 [==============================] - 0s 922us/step - loss: 106.9395
round 8600, game duration 4
1/1 [==============================] - 0s 1ms/step - loss: 4.4045
round 8700, game duration 4
1/1 [==============================] - 0s 861us/step - loss: 75.7940
round 8800, game duration 3
1/1 [==============================] - 0s 1ms/step - loss: 233.9208
round 8900, game duration 5
1/1 [==============================] - 0s 863us/step - loss: 56.2459
round 9000, game duration 4
1/1 [==============================] - 0s 1ms/step - loss: 78.6765
round 9100, game duration 4
1/1 [==============================] - 0s 865us/step - loss: 4.9784
round 9200, game duration 2
1/1 [==============================] - 0s 895us/step - loss: 40.0656
round 9300, game duration 6
1/1 [==============================] - 0s 902us/step - loss: 205.2079
round 9400, game duration 7
1/1 [=============================

In [6]:
def play_game(env, agent):
    done = False
    env.reset()
    env.render()
    while not done:
        action = agent.get_action(env)
        print("prediction", model.predict(np.array([env._one_hot_board()])))
        print("action:", action)
        obs,reward,done,info = env.step(action)
        env.render()

In [7]:
agent.set_use_randomness(False)
play_game(env, agent)

0 0 0
0 0 0
0 0 0
prediction [[104.399086 106.468605 107.04008  100.85167  117.147606 105.31688
   98.74773  113.27484   99.71636 ]]
action: 4
0 0 0
0 1 0
0 0 0
prediction [[157.175   160.42621 156.68333 158.6314  169.54083 166.26317 158.28264
  164.12338 156.46956]]
action: 4
0 0 0
0 1 0
0 0 0


In [8]:
play_game(env, agent)

0 0 0
0 0 0
0 0 0
prediction [[104.399086 106.468605 107.04008  100.85167  117.147606 105.31688
   98.74773  113.27484   99.71636 ]]
action: 4
0 0 0
0 1 0
0 0 0
prediction [[157.175   160.42621 156.68333 158.6314  169.54083 166.26317 158.28264
  164.12338 156.46956]]
action: 4
0 0 0
0 1 0
0 0 0
